In [28]:
# !pip install rgf_python

In [413]:
import os
import sys
import glob
import pathlib
import numpy as np
import pandas as pd

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from rgf.sklearn import RGFClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
# from sklearn.ensemble import BaggingClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as MAE

In [414]:
sys.path.append(os.path.abspath('../src/'))
    
from model_selection import cross_val_score

from data import data_generator

In [415]:
from quality_estimator import BaseQualityEstimator

In [416]:
rgf = RGFClassifier(max_leaf=400,
                    algorithm="RGF_Sib",
                    test_interval=100,
                    verbose=False)

### Check metrics:

In [614]:
X, Xy, y = data_generator(root_dir="../data/ellipse/")

In [615]:
q_clf = BaseQualityEstimator(metrics=["dice_coefficient", 
                                      "mae", "mse", 
#                                       "hausdorff_distance", 
#                                       "surface_distances",
#                                       "assd"
                                     ], 
                             unary_metrics=["area"],
                             matching_metrics=[
                                 "match2tpr",
#                                  "gt_match_aggregator",
#                                  "pred_match_aggregator",
                                 "match2predBinRate",
                                 "match2gtBinRate"
                             ],
                             meta_clf=LGBMClassifier(max_depth=2))

In [616]:
q_clf.fit(X, Xy, y)

BaseQualityEstimator(matching_metrics=['match2tpr', 'match2predBinRate',
                                       'match2gtBinRate'],
                     meta_clf=LGBMClassifier(max_depth=2))

In [618]:
X_meta = q_clf.X_metrics

In [619]:
X_meta.columns = [c_ + "_ellipse" for c_ in X_meta.columns]

In [15]:
# X_meta

In [527]:
# X_meta.to_csv("../data/X_meta_rectangular_full.csv", index=False)
# X_meta_rect = pd.read_csv("../data/X_meta_rectangular_full.csv").drop(columns=["area"])

In [545]:
X_meta = pd.read_csv("../data/X_meta_full.csv")
# X_meta_vovan = pd.read_csv("../data/X_meta_vovanminmin_full.csv").drop(columns=["area", 
#                                                                                 "dice_coefficient", "match2tpr"])

# X_meta = pd.concat([X_meta_artem, X_meta_vovan], axis=1)

In [418]:
from sklearn.model_selection import cross_val_predict

In [419]:
# q_clf.X_metrics

In [420]:
# !pip install xgboost

In [421]:
# X_meta_rect

In [546]:
X_meta_artem = pd.read_csv("../data/X_meta_full.csv").drop(columns=["area"])

In [547]:
# X_meta_rect.columns = [c_ + "_rect" for c_ in X_meta_rect.columns]

In [548]:
X_meta = pd.concat([X_meta_rect, X_meta_artem], axis=1)

In [549]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [550]:
# X_meta

In [551]:
cv = 15

In [552]:
from sklearn.model_selection import GroupKFold

In [553]:
cv = GroupKFold(n_splits=cv)

In [554]:
groups = list(range(len(y) // 3)) * 3

In [555]:
# groups

In [556]:
g = list(cv.split(X_meta, y, groups=groups))

In [557]:
# vals_lda_rect

In [558]:
vals_lgbm_rect = cross_val_predict(LGBMClassifier(max_depth=2), X_meta_rect, y, method='predict', cv=g)
vals_rgf_rect = cross_val_predict(rgf, X_meta_rect, y, method='predict', cv=g)
vals_rf_rect = cross_val_predict(RandomForestClassifier(n_estimators=100),
                                 X_meta_rect.fillna(0), y, method='predict', cv=g)
vals_xgb_rect = cross_val_predict(XGBClassifier(max_depth=3, n_estimators=10), 
                                  X_meta_rect, y, method='predict', cv=g)

In [559]:
vals_lgbm = cross_val_predict(LGBMClassifier(max_depth=2), X_meta_artem, y, method='predict', cv=g)
vals_rgf = cross_val_predict(rgf, X_meta_artem.fillna(0), y, method='predict', cv=g)
vals_rf = cross_val_predict(RandomForestClassifier(n_estimators=100), 
                            X_meta_artem.fillna(0), y, method='predict', cv=g)
vals_xgb = cross_val_predict(XGBClassifier(max_depth=3, n_estimators=10), 
                             X_meta_artem, y, method='predict', cv=g)

In [560]:
# vals_aggregated = (vals_lgbm + vals_rf + vals_xgb + vals_rgf + vals_lgbm_rect + 
#                    vals_rf_rect + vals_xgb_rect + vals_rgf_rect) / 8

# vals_aggregated = np.argmax(vals_aggregated, axis=1) + 1

In [561]:
# X_meta_artem = pd.read_csv("../data/X_meta_hausdorff_distance.csv")

In [562]:
vals_aggregated

array([1, 5, 5, 5, 4, 2, 2, 3, 2, 1, 4, 3, 5, 5, 4, 5, 1, 5, 4, 2, 3, 2,
       3, 4, 4, 3, 5, 5, 5, 3, 4, 5, 3, 2, 4, 1, 2, 3, 2, 2, 2, 3, 3, 2,
       4, 3, 5, 2, 5, 4, 3, 3, 5, 2, 4, 2, 1, 3, 3, 5, 5, 5, 5, 5, 3, 3,
       5, 3, 3, 5, 3, 1, 4, 2, 5, 1, 1, 2, 4, 4, 5, 3, 5, 2, 2, 5, 5, 3,
       5, 3, 1, 1, 3, 3, 4, 1, 1, 4, 2, 5, 2, 5, 3, 4, 5, 5, 5, 1, 3, 5,
       1, 4, 5, 5, 4, 1, 2, 3, 3, 1, 1, 1, 3, 2, 3, 1, 5, 3, 2, 1, 2, 1,
       4, 2, 3, 5, 1, 2, 2, 5, 4, 4, 5, 1, 3, 5, 5, 3, 3, 3, 2, 5, 3, 1,
       3, 1, 1, 2, 2, 5, 2, 3, 5, 2, 3, 5, 2, 1, 1, 3, 3, 3, 5, 1, 2, 5,
       1, 3, 2, 1])

In [563]:
vals_xgb

array([1, 5, 5, 5, 4, 2, 3, 3, 2, 1, 4, 2, 5, 5, 4, 5, 1, 5, 4, 3, 4, 2,
       3, 3, 4, 3, 5, 5, 5, 3, 3, 5, 3, 2, 4, 1, 2, 3, 2, 2, 2, 4, 3, 3,
       4, 3, 5, 2, 5, 4, 2, 3, 5, 2, 4, 3, 1, 3, 3, 5, 5, 5, 5, 5, 4, 3,
       5, 3, 3, 5, 3, 1, 4, 5, 5, 1, 1, 2, 4, 3, 5, 3, 5, 2, 2, 5, 5, 4,
       5, 3, 1, 1, 3, 3, 4, 1, 1, 3, 2, 5, 2, 5, 3, 3, 5, 5, 5, 3, 3, 5,
       1, 3, 5, 5, 4, 1, 3, 3, 3, 1, 1, 1, 3, 5, 3, 1, 5, 3, 3, 1, 2, 1,
       4, 2, 3, 5, 1, 5, 3, 5, 4, 3, 5, 1, 3, 5, 5, 3, 3, 3, 2, 5, 3, 1,
       3, 1, 1, 2, 2, 5, 2, 1, 5, 2, 4, 5, 2, 1, 1, 3, 3, 3, 5, 1, 2, 5,
       1, 3, 2, 1])

In [564]:
MAE(y, (vals_lgbm_rect + vals_lgbm) / 2)

0.4888888888888889

In [565]:
MAE(y, (vals_lgbm_rect) / 1)

0.5222222222222223

In [566]:
MAE(y, (vals_lgbm) / 1)

0.5111111111111111

In [567]:
MAE(y, (vals_lgbm) / 1)

0.5111111111111111

In [568]:
MAE(y, (vals_lgbm + vals_rgf) / 2)

0.48055555555555557

In [569]:
MAE(y, (vals_lgbm + vals_xgb + vals_rgf) / 3)

0.5

In [570]:
vals_all = (vals_lgbm + vals_rf + vals_xgb + vals_rgf + vals_lgbm_rect + 
            vals_rf_rect + vals_xgb_rect + vals_rgf_rect) / 8

In [571]:
MAE(y, vals_all)

0.49375

In [572]:
y_dummy = np.array([3 for i in range(len(y))])

In [573]:
MAE(y, y_dummy)

1.1777777777777778

In [612]:
y

array([1, 5, 4, 4, 3, 2, 2, 2, 3, 1, 3, 2, 5, 4, 4, 5, 1, 5, 4, 2, 5, 2,
       3, 3, 3, 3, 5, 1, 4, 4, 4, 5, 5, 2, 4, 1, 3, 4, 2, 2, 3, 2, 2, 4,
       5, 3, 5, 2, 5, 3, 3, 3, 4, 2, 4, 3, 1, 4, 4, 5, 5, 5, 4, 4, 5, 3,
       5, 3, 3, 4, 2, 1, 3, 3, 4, 4, 1, 3, 4, 2, 5, 2, 5, 3, 2, 5, 5, 3,
       5, 4, 1, 1, 5, 4, 3, 5, 1, 5, 4, 5, 2, 5, 2, 4, 5, 5, 5, 1, 3, 4,
       1, 3, 4, 5, 5, 1, 3, 5, 4, 1, 1, 3, 3, 2, 2, 1, 5, 3, 3, 1, 3, 1,
       3, 2, 3, 5, 1, 3, 2, 5, 5, 3, 5, 1, 2, 5, 5, 3, 3, 4, 4, 5, 4, 1,
       2, 1, 1, 2, 2, 4, 2, 5, 1, 2, 3, 5, 2, 1, 1, 4, 3, 3, 5, 1, 2, 5,
       1, 3, 3, 1])

In [613]:
vals_all.astype(int)

array([1, 4, 4, 4, 4, 2, 2, 2, 2, 1, 4, 2, 5, 4, 4, 5, 1, 5, 4, 2, 3, 2,
       3, 4, 4, 3, 5, 4, 5, 3, 3, 5, 3, 2, 4, 1, 2, 3, 2, 2, 2, 3, 3, 2,
       4, 3, 5, 2, 5, 4, 2, 3, 4, 2, 4, 2, 1, 3, 3, 5, 5, 5, 4, 5, 3, 2,
       4, 3, 2, 5, 3, 1, 4, 3, 4, 1, 1, 2, 3, 2, 5, 3, 5, 2, 2, 5, 5, 3,
       5, 3, 1, 1, 3, 3, 4, 1, 1, 3, 2, 5, 2, 5, 3, 3, 4, 5, 5, 2, 3, 5,
       1, 3, 4, 5, 4, 1, 2, 3, 3, 1, 1, 1, 3, 3, 3, 1, 5, 3, 2, 1, 2, 1,
       3, 2, 3, 5, 1, 2, 2, 5, 4, 3, 5, 1, 3, 5, 5, 3, 3, 3, 2, 5, 3, 1,
       2, 1, 1, 2, 2, 5, 2, 3, 4, 2, 3, 5, 2, 1, 1, 3, 3, 2, 5, 1, 2, 5,
       1, 3, 2, 1])